In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import scipy.stats as stats
import seaborn as sns

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, ExtraTreesClassifier, BaggingClassifier
from collections import Counter
from sklearn.metrics import roc_auc_score, mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

import xgboost as xgb #Gradient boosting

from xgboost import XGBClassifier
from sklearn.feature_selection import SelectFromModel #model selection
from sklearn.metrics import accuracy_score #model testing
import matplotlib.mlab as mlab  
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score, log_loss

from sklearn import decomposition

# Ensemble model for minute data

In [2]:
data=pd.read_csv('/Users/dingding/Desktop/GIII_3minutenew.csv')
data=data.drop(data.index[[0,1,2,3]])

null_columns=data.columns[data.isnull().any()]
data[null_columns].isnull().sum()
data=data.dropna(subset = ['MFI_3', 'Donchian_3'])
features=data.iloc[:,2:-2]
labels=data.iloc[:,-1]
import pandas as pd
from sklearn import preprocessing

min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(features)
features_normalized = pd.DataFrame(np_scaled)

In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingClassifier  #GBM algorithm
from sklearn import cross_validation, metrics   #Additional scklearn functions
from sklearn.grid_search import GridSearchCV   #Perforing grid search

from sklearn.cross_validation import KFold

from sklearn.cross_validation import StratifiedKFold

/Users/dingding/anaconda/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/dingding/anaconda/lib/python3.5/site-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


In [4]:
X_train, X_test, y_train, y_test = train_test_split(features_normalized, labels, test_size=0.2, random_state=143)

In [5]:
y_train
y_train.isnull().any()

False

# Grid search for XGBoost

In [116]:
param_test1 = {
 'max_depth':np.arange(3,10,2),
 'min_child_weight':np.arange(1,6,2),
    'n_estimators':np.arange(500,1600,500)
}
gsearch1 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test1, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch1.fit(X_train,y_train)
gsearch1.grid_scores_, gsearch1.best_params_, gsearch1.best_score_

([mean: 0.96812, std: 0.00184, params: {'n_estimators': 500, 'min_child_weight': 1, 'max_depth': 3},
  mean: 0.97048, std: 0.00168, params: {'n_estimators': 1000, 'min_child_weight': 1, 'max_depth': 3},
  mean: 0.97131, std: 0.00168, params: {'n_estimators': 1500, 'min_child_weight': 1, 'max_depth': 3},
  mean: 0.96798, std: 0.00194, params: {'n_estimators': 500, 'min_child_weight': 3, 'max_depth': 3},
  mean: 0.97020, std: 0.00171, params: {'n_estimators': 1000, 'min_child_weight': 3, 'max_depth': 3},
  mean: 0.97106, std: 0.00158, params: {'n_estimators': 1500, 'min_child_weight': 3, 'max_depth': 3},
  mean: 0.96820, std: 0.00175, params: {'n_estimators': 500, 'min_child_weight': 5, 'max_depth': 3},
  mean: 0.97044, std: 0.00141, params: {'n_estimators': 1000, 'min_child_weight': 5, 'max_depth': 3},
  mean: 0.97134, std: 0.00151, params: {'n_estimators': 1500, 'min_child_weight': 5, 'max_depth': 3},
  mean: 0.97196, std: 0.00142, params: {'n_estimators': 500, 'min_child_weight': 1, '

In [118]:
y_predicted = gsearch1.predict(X_test)
print(confusion_matrix(y_test, y_predicted))  
print(classification_report(y_test, y_predicted)) 
print (accuracy_score(y_test, y_predicted))

[[3474  395]
 [ 422 3934]]
             precision    recall  f1-score   support

         -1       0.89      0.90      0.89      3869
          1       0.91      0.90      0.91      4356

avg / total       0.90      0.90      0.90      8225

0.9006686930091186


/Users/dingding/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


# Grid search for GB

In [6]:
param_test2 = {
 'max_features':np.arange(3,9,2),
 'min_samples_split':np.arange(50,80,10),
    'n_estimators':np.arange(500,1600,500)
}
gsearch2 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.005, n_estimators=1500,max_depth=9, 
        min_samples_split=1200, min_samples_leaf=60, subsample=0.85, random_state=10, max_features=7,
warm_start=True), 
param_grid = param_test2, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch2.fit(X_train,y_train)
gsearch2.grid_scores_, gsearch2.best_params_, gsearch2.best_score_



([mean: 0.96132, std: 0.00261, params: {'max_features': 3, 'min_samples_split': 50, 'n_estimators': 500},
  mean: 0.96589, std: 0.00220, params: {'max_features': 3, 'min_samples_split': 50, 'n_estimators': 1000},
  mean: 0.96812, std: 0.00198, params: {'max_features': 3, 'min_samples_split': 50, 'n_estimators': 1500},
  mean: 0.96132, std: 0.00261, params: {'max_features': 3, 'min_samples_split': 60, 'n_estimators': 500},
  mean: 0.96589, std: 0.00220, params: {'max_features': 3, 'min_samples_split': 60, 'n_estimators': 1000},
  mean: 0.96812, std: 0.00198, params: {'max_features': 3, 'min_samples_split': 60, 'n_estimators': 1500},
  mean: 0.96132, std: 0.00261, params: {'max_features': 3, 'min_samples_split': 70, 'n_estimators': 500},
  mean: 0.96589, std: 0.00220, params: {'max_features': 3, 'min_samples_split': 70, 'n_estimators': 1000},
  mean: 0.96812, std: 0.00198, params: {'max_features': 3, 'min_samples_split': 70, 'n_estimators': 1500},
  mean: 0.96468, std: 0.00243, params: {

In [7]:

gb_predicted = gsearch2.predict(X_test)
print(confusion_matrix(y_test, gb_predicted))  
print(classification_report(y_test, gb_predicted))  
print (accuracy_score(y_test, gb_predicted))

[[3459  410]
 [ 428 3928]]
             precision    recall  f1-score   support

         -1       0.89      0.89      0.89      3869
          1       0.91      0.90      0.90      4356

avg / total       0.90      0.90      0.90      8225

0.8981155015197568


# Grid search for Random Forest

In [8]:
# Import the model we are using
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
param_test3 = {
 'max_features':np.arange(3,9,2),
 'min_samples_split':np.arange(50,80,10),
    'n_estimators':np.arange(500,1600,500)
}
gsearch3 = GridSearchCV(estimator = RandomForestClassifier(n_estimators = 10, max_depth=10, min_samples_split=10,max_features=5), 
param_grid = param_test3, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch3.fit(X_train,y_train)
gsearch3.grid_scores_, gsearch3.best_params_, gsearch3.best_score_




([mean: 0.95610, std: 0.00290, params: {'max_features': 3, 'min_samples_split': 50, 'n_estimators': 500},
  mean: 0.95601, std: 0.00294, params: {'max_features': 3, 'min_samples_split': 50, 'n_estimators': 1000},
  mean: 0.95598, std: 0.00292, params: {'max_features': 3, 'min_samples_split': 50, 'n_estimators': 1500},
  mean: 0.95562, std: 0.00302, params: {'max_features': 3, 'min_samples_split': 60, 'n_estimators': 500},
  mean: 0.95570, std: 0.00301, params: {'max_features': 3, 'min_samples_split': 60, 'n_estimators': 1000},
  mean: 0.95570, std: 0.00295, params: {'max_features': 3, 'min_samples_split': 60, 'n_estimators': 1500},
  mean: 0.95541, std: 0.00286, params: {'max_features': 3, 'min_samples_split': 70, 'n_estimators': 500},
  mean: 0.95536, std: 0.00301, params: {'max_features': 3, 'min_samples_split': 70, 'n_estimators': 1000},
  mean: 0.95544, std: 0.00290, params: {'max_features': 3, 'min_samples_split': 70, 'n_estimators': 1500},
  mean: 0.96029, std: 0.00257, params: {

In [9]:
# Use the forest's predict method on the test data
predictions = gsearch3.predict(X_test)
# Calculate the absolute errors
print(confusion_matrix(y_test, predictions))  
print(classification_report(y_test, predictions))  
print (accuracy_score(y_test, predictions))

[[3383  486]
 [ 476 3880]]
             precision    recall  f1-score   support

         -1       0.88      0.87      0.88      3869
          1       0.89      0.89      0.89      4356

avg / total       0.88      0.88      0.88      8225

0.8830395136778115


# Grid search for Extra Trees

In [118]:
param_test12 = {
 'max_features':np.arange(3,9,2),
 'min_samples_split':np.arange(50,80,10),
    'n_estimators':np.arange(500,1600,500)
}
gsearch12 = GridSearchCV(estimator = ExtraTreesClassifier(n_estimators=5,criterion= 'entropy',min_samples_split= 5,
                            max_depth= 50, min_samples_leaf=5), 
param_grid = param_test12, scoring='roc_auc',n_jobs=4,iid=False, cv=5)   
gsearch12.fit(X_train, y_train)
gsearch12.grid_scores_, gsearch12.best_params_, gsearch12.best_score_


extc_pred = gsearch12.predict(X_test)

In [119]:
print(confusion_matrix(y_test, extc_pred))  
print(classification_report(y_test, extc_pred))  
print (accuracy_score(y_test, extc_pred))


[[3239  630]
 [ 472 3884]]
             precision    recall  f1-score   support

         -1       0.87      0.84      0.85      3869
          1       0.86      0.89      0.88      4356

avg / total       0.87      0.87      0.87      8225

0.8660182370820668


# Ensemble model for daily data

In [157]:
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
import urllib.request, json
import os
import numpy as np
import tensorflow as tf # This code has been tested with TensorFlow 1.6
from sklearn.preprocessing import MinMaxScaler
from keras.models import Sequential
from keras.layers.core import Dense, Activation, Dropout
from keras.layers.recurrent import LSTM
import time



In [104]:
X, X_test, y, y_test = train_test_split(features_normalized, labels, test_size = 0.20) 
X.shape[1]

15

In [11]:
daily_data=pd.read_csv('/Users/dingding/Desktop/GIIIUS_daily_new.csv')
null_columns=daily_data.columns[daily_data.isnull().any()]
daily_data[null_columns].isnull().sum()
daily_data=daily_data.dropna(subset = ['Donchian_3'])
daily_data=daily_data.drop(['field','RETURN_COM_EQY','EBITDA','IS_EPS','PROF_MARGIN','ROC_3','Return_3','return/ebitda','return/EPS','return/pro','return/ROE'],axis = 1)
daily_features=daily_data.iloc[:,0:-1]
daily_labels=daily_data.iloc[:,-1]
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(daily_features)
dailyfeatures_normalized = pd.DataFrame(np_scaled)
(daily_features == 0).sum(axis=0)

high               0
close              0
low                0
open               0
volume             0
CUR_MKT_CAP        0
PE_RATIO           0
MA_3               0
Trix_3             0
MACD_1_4           0
RSI_3              0
MFI_3           1002
OBV_3             60
Force_3          423
STD_3              0
Donchian_3        20
cap/ebitda         0
close/ebitda       0
cap/EPS            0
close/EPS          0
cap/ROE            0
close/ROE          0
cap/pro            0
close/pro          0
dtype: int64

# Grid search for XGBoost

In [14]:
X_dailytrain, X_dailytest, y_dailytrain, y_dailytest = train_test_split(dailyfeatures_normalized, daily_labels, test_size = 0.20)  
param_test5 = {
 'max_depth':np.arange(3,10,2),
 'min_child_weight':np.arange(1,6,2),
    'n_estimators':np.arange(500,1600,500)
}
gsearch5 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test5, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch5.fit(X_dailytrain,y_dailytrain)
gsearch5.grid_scores_, gsearch5.best_params_, gsearch5.best_score_

([mean: 0.94365, std: 0.00281, params: {'max_depth': 3, 'n_estimators': 500, 'min_child_weight': 1},
  mean: 0.94090, std: 0.00244, params: {'max_depth': 3, 'n_estimators': 1000, 'min_child_weight': 1},
  mean: 0.93919, std: 0.00299, params: {'max_depth': 3, 'n_estimators': 1500, 'min_child_weight': 1},
  mean: 0.94412, std: 0.00293, params: {'max_depth': 3, 'n_estimators': 500, 'min_child_weight': 3},
  mean: 0.94046, std: 0.00214, params: {'max_depth': 3, 'n_estimators': 1000, 'min_child_weight': 3},
  mean: 0.93877, std: 0.00275, params: {'max_depth': 3, 'n_estimators': 1500, 'min_child_weight': 3},
  mean: 0.94284, std: 0.00318, params: {'max_depth': 3, 'n_estimators': 500, 'min_child_weight': 5},
  mean: 0.94019, std: 0.00283, params: {'max_depth': 3, 'n_estimators': 1000, 'min_child_weight': 5},
  mean: 0.93837, std: 0.00321, params: {'max_depth': 3, 'n_estimators': 1500, 'min_child_weight': 5},
  mean: 0.94234, std: 0.00326, params: {'max_depth': 5, 'n_estimators': 500, 'min_chi

In [15]:
y_predicted = gsearch5.predict(X_dailytest)

/Users/dingding/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [16]:

from sklearn.metrics import classification_report, confusion_matrix  
print(confusion_matrix(y_dailytest, y_predicted))  
print(classification_report(y_dailytest, y_predicted)) 
print (accuracy_score(y_dailytest, y_predicted))

[[502  97]
 [ 88 631]]
             precision    recall  f1-score   support

         -1       0.85      0.84      0.84       599
          1       0.87      0.88      0.87       719

avg / total       0.86      0.86      0.86      1318

0.8596358118361154


# Grid search for GB

In [17]:
param_test6 = {
 'max_features':np.arange(3,9,2),
 'min_samples_split':np.arange(50,80,10),
    'n_estimators':np.arange(500,1600,500)
}
gsearch6 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.005, n_estimators=1500,max_depth=9, 
        min_samples_split=1200, min_samples_leaf=60, subsample=0.85, random_state=10, max_features=7,
warm_start=True), 
param_grid = param_test6, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch6.fit(X_dailytrain,y_dailytrain)
gsearch6.grid_scores_, gsearch6.best_params_, gsearch6.best_score_



([mean: 0.92901, std: 0.00560, params: {'max_features': 3, 'min_samples_split': 50, 'n_estimators': 500},
  mean: 0.93744, std: 0.00483, params: {'max_features': 3, 'min_samples_split': 50, 'n_estimators': 1000},
  mean: 0.94081, std: 0.00411, params: {'max_features': 3, 'min_samples_split': 50, 'n_estimators': 1500},
  mean: 0.92901, std: 0.00560, params: {'max_features': 3, 'min_samples_split': 60, 'n_estimators': 500},
  mean: 0.93744, std: 0.00483, params: {'max_features': 3, 'min_samples_split': 60, 'n_estimators': 1000},
  mean: 0.94081, std: 0.00411, params: {'max_features': 3, 'min_samples_split': 60, 'n_estimators': 1500},
  mean: 0.92901, std: 0.00560, params: {'max_features': 3, 'min_samples_split': 70, 'n_estimators': 500},
  mean: 0.93744, std: 0.00483, params: {'max_features': 3, 'min_samples_split': 70, 'n_estimators': 1000},
  mean: 0.94081, std: 0.00411, params: {'max_features': 3, 'min_samples_split': 70, 'n_estimators': 1500},
  mean: 0.93536, std: 0.00526, params: {

In [18]:
GBpredict = gsearch6.predict(X_dailytest)
print(confusion_matrix(y_dailytest, GBpredict))  
print(classification_report(y_dailytest, GBpredict))  
print (accuracy_score(y_dailytest, GBpredict))

[[505  94]
 [ 78 641]]
             precision    recall  f1-score   support

         -1       0.87      0.84      0.85       599
          1       0.87      0.89      0.88       719

avg / total       0.87      0.87      0.87      1318

0.8694992412746586


# Grid search for Random Forest

In [19]:
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
param_test7 = {
 'max_features':np.arange(3,9,2),
 'min_samples_split':np.arange(50,80,10),
    'n_estimators':np.arange(500,1600,500)
}
gsearch7 = GridSearchCV(estimator = RandomForestClassifier(n_estimators = 10, max_depth=10, min_samples_split=10,max_features=5), 
param_grid = param_test7, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch7.fit(X_dailytrain,y_dailytrain)
gsearch7.grid_scores_, gsearch7.best_params_, gsearch7.best_score_


predict = gsearch7.predict(X_dailytest)
# Calculate the absolute errors


In [20]:
print(confusion_matrix(y_dailytest, predict))  
print(classification_report(y_dailytest, predict))  
print (accuracy_score(y_dailytest, GBpredict))

[[497 102]
 [ 86 633]]
             precision    recall  f1-score   support

         -1       0.85      0.83      0.84       599
          1       0.86      0.88      0.87       719

avg / total       0.86      0.86      0.86      1318

0.8694992412746586


In [95]:
extc = ExtraTreesClassifier(n_estimators=5,criterion= 'entropy',min_samples_split= 5,
                            max_depth= 50, min_samples_leaf=5
                           )      

extc.fit(X_dailytrain, y_dailytrain) 

print('Predict...')
extc_pred = extc.predict(X_dailytest)
print(confusion_matrix(y_dailytest, extc_pred))  
print(classification_report(y_dailytest, extc_pred))

Predict...
[[439 136]
 [149 594]]
             precision    recall  f1-score   support

         -1       0.75      0.76      0.75       575
          1       0.81      0.80      0.81       743

avg / total       0.78      0.78      0.78      1318



# Grid search for Extra Trees

In [115]:
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
param_test12 = {
 'max_features':np.arange(3,9,2),
 'min_samples_split':np.arange(50,80,10),
    'n_estimators':np.arange(500,1600,500)
}
gsearch12 = GridSearchCV(estimator = ExtraTreesClassifier(n_estimators=5,criterion= 'entropy',min_samples_split= 5,
                            max_depth= 50, min_samples_leaf=5), 
param_grid = param_test12, scoring='roc_auc',n_jobs=4,iid=False, cv=5)   
gsearch12.fit(X_dailytrain, y_dailytrain)
gsearch12.grid_scores_, gsearch12.best_params_, gsearch12.best_score_


extc_pred = gsearch12.predict(X_dailytest)

In [117]:
print(confusion_matrix(y_dailytest, extc_pred))  
print(classification_report(y_dailytest, extc_pred))
print (accuracy_score(y_dailytest, extc_pred))

[[466 133]
 [ 92 627]]
             precision    recall  f1-score   support

         -1       0.84      0.78      0.81       599
          1       0.82      0.87      0.85       719

avg / total       0.83      0.83      0.83      1318

0.8292867981790591


# miunte data with feature importance

In [44]:
data_feature=data.drop(['time','close','high','low','open','volume','MA_3','Force_3'],axis = 1)
features_importance=data_feature.iloc[:,2:-2]

labels_importance=data_feature.iloc[:,-1]
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(features_importance)
featuresimport_normalized = pd.DataFrame(np_scaled)

X_impor, X_impor_test, y_impor, y_impor_test = train_test_split(featuresimport_normalized, labels_importance, test_size = 0.20) 


In [37]:

xgbModel= XGBClassifier( learning_rate =0.1, n_estimators=500, max_depth=9,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27)
xgbModel = xgbModel.fit(X_impor,y_impor)
 
y_predicted = xgbModel.predict(X_impor_test)

/Users/dingding/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [38]:
print(confusion_matrix(y_impor_test, y_predicted))  
print(classification_report(y_impor_test, y_predicted))

[[3121  728]
 [ 551 3825]]
             precision    recall  f1-score   support

         -1       0.85      0.81      0.83      3849
          1       0.84      0.87      0.86      4376

avg / total       0.84      0.84      0.84      8225



In [50]:
gb= GradientBoostingClassifier(learning_rate=0.005, n_estimators=1500,max_depth=7, 
        min_samples_split=50, min_samples_leaf=60, subsample=0.85, random_state=10, max_features=5,
warm_start=True)
gb.fit(X_impor,y_impor)

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.005, loss='deviance', max_depth=7,
              max_features=5, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=60, min_samples_split=50,
              min_weight_fraction_leaf=0.0, n_estimators=1500,
              presort='auto', random_state=10, subsample=0.85, verbose=0,
              warm_start=True)

In [51]:

y_gb = gb.predict(X_impor_test)
print(confusion_matrix(y_impor_test, y_gb))  
print(classification_report(y_impor_test, y_gb))
print (accuracy_score(y_impor_test, y_gb))

[[3160  698]
 [ 546 3821]]
             precision    recall  f1-score   support

         -1       0.85      0.82      0.84      3858
          1       0.85      0.87      0.86      4367

avg / total       0.85      0.85      0.85      8225

0.8487537993920973


In [54]:
rf=RandomForestClassifier(n_estimators = 1000, max_depth=7, min_samples_split=50,max_features=5)
rf.fit(X_impor,y_impor)
y_rf = rf.predict(X_impor_test)
print(confusion_matrix(y_impor_test, y_rf))  
print(classification_report(y_impor_test, y_rf))
print (accuracy_score(y_impor_test, y_rf))

[[3106  752]
 [ 578 3789]]
             precision    recall  f1-score   support

         -1       0.84      0.81      0.82      3858
          1       0.83      0.87      0.85      4367

avg / total       0.84      0.84      0.84      8225

0.8382978723404255


In [80]:
extc = ExtraTreesClassifier(n_estimators=5,criterion= 'entropy',min_samples_split= 5,
                            max_depth= 50, min_samples_leaf=5
                           )      

extc.fit(X_impor,y_impor) 

print('Predict...')
extc_pred = extc.predict(X_impor_test)
print(confusion_matrix(y_impor_test, extc_pred))  
print(classification_report(y_impor_test, extc_pred))
print (accuracy_score(y_impor_test, extc_pred))

Predict...
[[2984  874]
 [ 623 3744]]
             precision    recall  f1-score   support

         -1       0.83      0.77      0.80      3858
          1       0.81      0.86      0.83      4367

avg / total       0.82      0.82      0.82      8225

0.8179939209726443


# daily data with feature importance

In [67]:
daily_data=pd.read_csv('/Users/dingding/Desktop/GIIIUS_daily_new.csv')
null_columns=daily_data.columns[daily_data.isnull().any()]
daily_data[null_columns].isnull().sum()
daily_data=daily_data.dropna(subset = ['Donchian_3'])
daily_data_impor=daily_data.drop(['field','high','close','low','open','volume','CUR_MKT_CAP','PE_RATIO','RETURN_COM_EQY','EBITDA','IS_EPS','PROF_MARGIN','ROC_3','Return_3',
                                  'cap/ebitda','close/ebitda','cap/EPS','close/EPS','cap/ROE',
                                  'close/ROE','cap/pro','close/pro','return/ebitda','return/EPS','return/pro','return/ROE','MFI_3'],axis = 1)
daily_features_impor=daily_data_impor.iloc[:,0:-1]
daily_labels_impor=daily_data_impor.iloc[:,-1]
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(daily_features_impor)
daily_featuresimport_normalized = pd.DataFrame(np_scaled)

Xdaily_impor, Xdaily_impor_test, ydaily_impor, ydaily_impor_test = train_test_split(daily_featuresimport_normalized, daily_labels_impor, test_size = 0.20)  


In [68]:
param_test9 = {
 'max_depth':np.arange(3,10,2),
 'min_child_weight':np.arange(1,6,2),
    'n_estimators':np.arange(500,1600,500)
}
gsearch9 = GridSearchCV(estimator = XGBClassifier( learning_rate =0.1, n_estimators=140, max_depth=5,
 min_child_weight=1, gamma=0, subsample=0.8, colsample_bytree=0.8,
 objective= 'binary:logistic', nthread=4, scale_pos_weight=1, seed=27), 
 param_grid = param_test9, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch9.fit(Xdaily_impor,ydaily_impor)
gsearch9.grid_scores_, gsearch9.best_params_, gsearch9.best_score_

([mean: 0.94599, std: 0.00537, params: {'max_depth': 3, 'n_estimators': 500, 'min_child_weight': 1},
  mean: 0.94387, std: 0.00510, params: {'max_depth': 3, 'n_estimators': 1000, 'min_child_weight': 1},
  mean: 0.94275, std: 0.00483, params: {'max_depth': 3, 'n_estimators': 1500, 'min_child_weight': 1},
  mean: 0.94606, std: 0.00664, params: {'max_depth': 3, 'n_estimators': 500, 'min_child_weight': 3},
  mean: 0.94401, std: 0.00625, params: {'max_depth': 3, 'n_estimators': 1000, 'min_child_weight': 3},
  mean: 0.94281, std: 0.00650, params: {'max_depth': 3, 'n_estimators': 1500, 'min_child_weight': 3},
  mean: 0.94614, std: 0.00679, params: {'max_depth': 3, 'n_estimators': 500, 'min_child_weight': 5},
  mean: 0.94433, std: 0.00664, params: {'max_depth': 3, 'n_estimators': 1000, 'min_child_weight': 5},
  mean: 0.94323, std: 0.00614, params: {'max_depth': 3, 'n_estimators': 1500, 'min_child_weight': 5},
  mean: 0.94658, std: 0.00544, params: {'max_depth': 5, 'n_estimators': 500, 'min_chi

In [69]:
xg_pred = gsearch9.predict(Xdaily_impor_test)
print(confusion_matrix(ydaily_impor_test, xg_pred))  
print(classification_report(ydaily_impor_test, xg_pred))
print (accuracy_score(ydaily_impor_test, xg_pred))

[[543  87]
 [ 78 610]]
             precision    recall  f1-score   support

         -1       0.87      0.86      0.87       630
          1       0.88      0.89      0.88       688

avg / total       0.87      0.87      0.87      1318

0.8748103186646434


/Users/dingding/anaconda/lib/python3.5/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [111]:
param_test10 = {
 'max_features':np.arange(3,9,2),
 'min_samples_split':np.arange(50,80,10),
    'n_estimators':np.arange(500,1600,500)
}
gsearch10 = GridSearchCV(estimator = GradientBoostingClassifier(learning_rate=0.005, n_estimators=1500,max_depth=9, 
        min_samples_split=1200, min_samples_leaf=60, subsample=0.85, random_state=10, max_features=7,
warm_start=True), 
param_grid = param_test10, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch10.fit(Xdaily_impor,ydaily_impor)
gsearch10.grid_scores_, gsearch10.best_params_, gsearch10.best_score_

([mean: 0.94124, std: 0.00474, params: {'max_features': 3, 'min_samples_split': 50, 'n_estimators': 500},
  mean: 0.94640, std: 0.00503, params: {'max_features': 3, 'min_samples_split': 50, 'n_estimators': 1000},
  mean: 0.94827, std: 0.00520, params: {'max_features': 3, 'min_samples_split': 50, 'n_estimators': 1500},
  mean: 0.94124, std: 0.00474, params: {'max_features': 3, 'min_samples_split': 60, 'n_estimators': 500},
  mean: 0.94640, std: 0.00503, params: {'max_features': 3, 'min_samples_split': 60, 'n_estimators': 1000},
  mean: 0.94827, std: 0.00520, params: {'max_features': 3, 'min_samples_split': 60, 'n_estimators': 1500},
  mean: 0.94124, std: 0.00474, params: {'max_features': 3, 'min_samples_split': 70, 'n_estimators': 500},
  mean: 0.94640, std: 0.00503, params: {'max_features': 3, 'min_samples_split': 70, 'n_estimators': 1000},
  mean: 0.94827, std: 0.00520, params: {'max_features': 3, 'min_samples_split': 70, 'n_estimators': 1500},
  mean: 0.94337, std: 0.00422, params: {

In [112]:
g_pred = gsearch10.predict(Xdaily_impor_test)
print(confusion_matrix(ydaily_impor_test, g_pred))  
print(classification_report(ydaily_impor_test, g_pred))
print (accuracy_score(ydaily_impor_test, g_pred))

[[558  72]
 [ 81 607]]
             precision    recall  f1-score   support

         -1       0.87      0.89      0.88       630
          1       0.89      0.88      0.89       688

avg / total       0.88      0.88      0.88      1318

0.8839150227617603


In [72]:
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
param_test11 = {
 'max_features':np.arange(3,9,2),
 'min_samples_split':np.arange(50,80,10),
    'n_estimators':np.arange(500,1600,500)
}
gsearch11 = GridSearchCV(estimator = RandomForestClassifier(n_estimators = 10, max_depth=10, min_samples_split=10,max_features=5), 
param_grid = param_test11, scoring='roc_auc',n_jobs=4,iid=False, cv=5)
gsearch11.fit(Xdaily_impor,ydaily_impor)
gsearch11.grid_scores_, gsearch11.best_params_, gsearch11.best_score_


predict = gsearch11.predict(Xdaily_impor_test)
# Calculate the absolute errors

In [73]:

print(confusion_matrix(ydaily_impor_test, predict))  
print(classification_report(ydaily_impor_test, predict))
print (accuracy_score(ydaily_impor_test, predict))

[[548  82]
 [ 79 609]]
             precision    recall  f1-score   support

         -1       0.87      0.87      0.87       630
          1       0.88      0.89      0.88       688

avg / total       0.88      0.88      0.88      1318

0.877845220030349


In [99]:
extc = ExtraTreesClassifier(n_estimators=5,criterion= 'entropy',min_samples_split= 5,
                            max_depth= 50, min_samples_leaf=5
                           )      

extc.fit(Xdaily_impor,ydaily_impor) 

print('Predict...')
extc_pred = extc.predict(Xdaily_impor_test)
print(confusion_matrix(ydaily_impor_test, extc_pred))  
print(classification_report(ydaily_impor_test, extc_pred))
print (accuracy_score(ydaily_impor_test, extc_pred))

Predict...
[[437 193]
 [ 78 610]]
             precision    recall  f1-score   support

         -1       0.85      0.69      0.76       630
          1       0.76      0.89      0.82       688

avg / total       0.80      0.79      0.79      1318

0.7943854324734446


In [108]:
extc = ExtraTreesClassifier(n_estimators=5,criterion= 'entropy',min_samples_split= 5,
                            max_depth= 50, min_samples_leaf=5
                           )      

extc.fit(Xdaily_impor,ydaily_impor) 

print('Predict...')
extc_pred = extc.predict(Xdaily_impor_test)
print(confusion_matrix(ydaily_impor_test, extc_pred))  
print(classification_report(ydaily_impor_test, extc_pred))
print (accuracy_score(ydaily_impor_test, extc_pred))

Predict...
[[443 187]
 [ 74 614]]
             precision    recall  f1-score   support

         -1       0.86      0.70      0.77       630
          1       0.77      0.89      0.82       688

avg / total       0.81      0.80      0.80      1318

0.8019726858877086


In [113]:
from sklearn.ensemble import RandomForestClassifier
# Instantiate model with 1000 decision trees
param_test12 = {
 'max_features':np.arange(3,9,2),
 'min_samples_split':np.arange(50,80,10),
    'n_estimators':np.arange(500,1600,500)
}
gsearch12 = GridSearchCV(estimator = ExtraTreesClassifier(n_estimators=5,criterion= 'entropy',min_samples_split= 5,
                            max_depth= 50, min_samples_leaf=5), 
param_grid = param_test12, scoring='roc_auc',n_jobs=4,iid=False, cv=5)   
gsearch12.fit(Xdaily_impor,ydaily_impor)
gsearch12.grid_scores_, gsearch12.best_params_, gsearch12.best_score_


extc_pred = gsearch12.predict(Xdaily_impor_test)
# Calculate the absolute errors

In [114]:

print(confusion_matrix(ydaily_impor_test, extc_pred))  
print(classification_report(ydaily_impor_test, extc_pred))
print (accuracy_score(ydaily_impor_test, extc_pred))

[[499 131]
 [ 68 620]]
             precision    recall  f1-score   support

         -1       0.88      0.79      0.83       630
          1       0.83      0.90      0.86       688

avg / total       0.85      0.85      0.85      1318

0.8490136570561456
